In [31]:
%%capture
!pip install -q unsloth
!pip install -q transformers==4.56.2 trl==0.22.2
!pip install -q jiwer einops addict easydict

In [32]:
from huggingface_hub import snapshot_download
snapshot_download("unsloth/DeepSeek-OCR", local_dir = "deepseek_ocr")

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

'/kaggle/working/deepseek_ocr'

In [33]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from transformers import AutoModel
import os
os.environ["UNSLOTH_WARN_UNINITIALIZED"] = '0'
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Qwen3-VL-8B-Instruct-bnb-4bit", # Qwen 3 vision support
    "unsloth/Qwen3-VL-8B-Thinking-bnb-4bit",
    "unsloth/Qwen3-VL-32B-Instruct-bnb-4bit",
    "unsloth/Qwen3-VL-32B-Thinking-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "./deepseek_ocr",
    load_in_4bit = False, # Use 4bit to reduce memory use. False for 16bit LoRA.
    auto_model = AutoModel,
    trust_remote_code=True,
    unsloth_force_compile=True,
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.12.9: Fast Deepseekocr patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at ./deepseek_ocr and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load dataset

In [34]:
import os, json
from datasets import Dataset

def load_hwdb_split(
    dataset_dir,
    split="train_data",
):
    split_dir = os.path.join(dataset_dir, split)
    # print("#1", split_dir)
    # print("#2", os.popen('ls /kaggle/input/Vietnamese_Handwritten_OCR').read())
    if not os.path.isdir(split_dir):
        raise FileNotFoundError(f"Không tìm thấy thư mục {split_dir}")

    samples = []
    subfolders = sorted(os.listdir(split_dir))

    for sub in subfolders:
        sub_path = os.path.join(split_dir, sub)
        label_path = os.path.join(sub_path, "label.json")

        if not os.path.isdir(sub_path) or not os.path.exists(label_path):
            continue

        with open(label_path, "r", encoding="utf-8-sig") as f:
            labels = json.load(f)

        for img_name, text in labels.items():
            img_path = os.path.join(sub_path, img_name)
            if not os.path.exists(img_path):
                continue

            samples.append({
                "image_path": img_path,
                "text": text.strip()
            })

    return Dataset.from_list(samples)

In [35]:
BASE_DIR = "/kaggle/input/Vietnamese_Handwritten_OCR"

LINE_DIR = os.path.join(
    BASE_DIR,
    "UIT_HWDB_line",
    "UIT_HWDB_line"
)

PARA_DIR = os.path.join(
    BASE_DIR,
    "UIT_HWDB_paragraph",
    "UIT_HWDB_paragraph"
)

ds_line = load_hwdb_split(
    dataset_dir=LINE_DIR,
    split="train_data"
)

ds_para = load_hwdb_split(
    dataset_dir=PARA_DIR,
    split="train_data"
)

from datasets import concatenate_datasets

dataset = concatenate_datasets([ds_line, ds_para])

print(dataset)
print("Total samples:", len(dataset))

Dataset({
    features: ['image_path', 'text'],
    num_rows: 8141
})
Total samples: 8141


Cài đặt các tham số cho model

In [36]:
model = FastVisionModel.get_peft_model(
    model,
    target_modules = [
        "q_proj",
        "v_proj",
        "o_proj",
    ],

    r = 8,       
    lora_alpha = 16, 
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False, 
    loftq_config = None,
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.model` require gradients


In [37]:
from PIL import Image

instruction = "<image>\nHãy chuyển đổi hình ảnh văn bản viết tay này sang định dạng text tiếng Việt chính xác."

def convert_to_conversation(sample):
    image = Image.open(sample["image_path"]).convert("RGB")
    return {
        "messages": [
            {
                "role": "<|User|>",
                "content": instruction,
                "images": [image],
            },
            {
                "role": "<|Assistant|>",
                "content": sample["text"],
            }
        ]
    }

In [38]:
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

Sắp xếp data

In [39]:
import torch
import math
from dataclasses import dataclass
from typing import Dict, List, Any, Tuple
from PIL import Image, ImageOps
from torch.nn.utils.rnn import pad_sequence
import io

from deepseek_ocr.modeling_deepseekocr import (
    format_messages,
    text_encode,
    BasicImageTransform,
    dynamic_preprocess,
)

@dataclass
class DeepSeekOCRDataCollator:
    """
    Args:
        tokenizer: Tokenizer
        model: Model
        image_size: Size for image patches (default: 640)
        base_size: Size for global view (default: 1024)
        crop_mode: Whether to use dynamic cropping for large images
        train_on_responses_only: If True, only train on assistant responses (mask user prompts)
    """
    tokenizer: Any
    model: Any
    image_size: int = 640
    base_size: int = 1024
    crop_mode: bool = True
    image_token_id: int = 128815
    train_on_responses_only: bool = True

    def __init__(
        self,
        tokenizer,
        model,
        image_size: int = 640,
        base_size: int = 1024,
        crop_mode: bool = True,
        train_on_responses_only: bool = True,
    ):
        self.tokenizer = tokenizer
        self.model = model
        self.image_size = image_size
        self.base_size = base_size
        self.crop_mode = crop_mode
        self.image_token_id = 128815
        self.dtype = model.dtype  # Get dtype from model
        self.train_on_responses_only = train_on_responses_only

        self.image_transform = BasicImageTransform(
            mean=(0.5, 0.5, 0.5),
            std=(0.5, 0.5, 0.5),
            normalize=True
        )
        self.patch_size = 16
        self.downsample_ratio = 4

        # Get BOS token ID from tokenizer
        if hasattr(tokenizer, 'bos_token_id') and tokenizer.bos_token_id is not None:
            self.bos_id = tokenizer.bos_token_id
        else:
            self.bos_id = 0
            print(f"Warning: tokenizer has no bos_token_id, using default: {self.bos_id}")

    def deserialize_image(self, image_data) -> Image.Image:
        """Convert image data (bytes dict or PIL Image) to PIL Image in RGB mode"""
        if isinstance(image_data, Image.Image):
            return image_data.convert("RGB")
        elif isinstance(image_data, dict) and 'bytes' in image_data:
            image_bytes = image_data['bytes']
            image = Image.open(io.BytesIO(image_bytes))
            return image.convert("RGB")
        else:
            raise ValueError(f"Unsupported image format: {type(image_data)}")

    def calculate_image_token_count(self, image: Image.Image, crop_ratio: Tuple[int, int]) -> int:
        """Calculate the number of tokens this image will generate"""
        num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
        num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

        width_crop_num, height_crop_num = crop_ratio

        if self.crop_mode:
            img_tokens = num_queries_base * num_queries_base + 1
            if width_crop_num > 1 or height_crop_num > 1:
                img_tokens += (num_queries * width_crop_num + 1) * (num_queries * height_crop_num)
        else:
            img_tokens = num_queries * num_queries + 1

        return img_tokens

    def process_image(self, image: Image.Image) -> Tuple[List, List, List, List, Tuple[int, int]]:
        """
        Process a single image based on crop_mode and size thresholds

        Returns:
            Tuple of (images_list, images_crop_list, images_spatial_crop, tokenized_image, crop_ratio)
        """
        images_list = []
        images_crop_list = []
        images_spatial_crop = []

        if self.crop_mode:
            # Determine crop ratio based on image size
            if image.size[0] <= 640 and image.size[1] <= 640:
                crop_ratio = (1, 1)
                images_crop_raw = []
            else:
                images_crop_raw, crop_ratio = dynamic_preprocess(
                    image, min_num=2, max_num=9,
                    image_size=self.image_size, use_thumbnail=False
                )

            # Process global view with padding
            global_view = ImageOps.pad(
                image, (self.base_size, self.base_size),
                color=tuple(int(x * 255) for x in self.image_transform.mean)
            )
            images_list.append(self.image_transform(global_view).to(self.dtype))

            width_crop_num, height_crop_num = crop_ratio
            images_spatial_crop.append([width_crop_num, height_crop_num])

            # Process local views (crops) if applicable
            if width_crop_num > 1 or height_crop_num > 1:
                for crop_img in images_crop_raw:
                    images_crop_list.append(
                        self.image_transform(crop_img).to(self.dtype)
                    )

            # Calculate image tokens
            num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
            num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

            tokenized_image = ([self.image_token_id] * num_queries_base + [self.image_token_id]) * num_queries_base
            tokenized_image += [self.image_token_id]

            if width_crop_num > 1 or height_crop_num > 1:
                tokenized_image += ([self.image_token_id] * (num_queries * width_crop_num) + [self.image_token_id]) * (
                    num_queries * height_crop_num)

        else:  # crop_mode = False
            crop_ratio = (1, 1)
            images_spatial_crop.append([1, 1])

            # For smaller base sizes, resize; for larger, pad
            if self.base_size <= 640:
                resized_image = image.resize((self.base_size, self.base_size), Image.LANCZOS)
                images_list.append(self.image_transform(resized_image).to(self.dtype))
            else:
                global_view = ImageOps.pad(
                    image, (self.base_size, self.base_size),
                    color=tuple(int(x * 255) for x in self.image_transform.mean)
                )
                images_list.append(self.image_transform(global_view).to(self.dtype))

            num_queries = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)
            tokenized_image = ([self.image_token_id] * num_queries + [self.image_token_id]) * num_queries
            tokenized_image += [self.image_token_id]

        return images_list, images_crop_list, images_spatial_crop, tokenized_image, crop_ratio

    def process_single_sample(self, messages: List[Dict]) -> Dict[str, Any]:
            """
            Process a single conversation into model inputs.
            """

            # --- 1. Setup ---
            images = []
            for message in messages:
                if "images" in message and message["images"]:
                    for img_data in message["images"]:
                        if img_data is not None:
                            pil_image = self.deserialize_image(img_data)
                            images.append(pil_image)

            if not images:
                raise ValueError("No images found in sample. Please ensure all samples contain images.")

            tokenized_str = []
            images_seq_mask = []
            images_list, images_crop_list, images_spatial_crop = [], [], []

            prompt_token_count = -1 # Index to start training
            assistant_started = False
            image_idx = 0

            # Add BOS token at the very beginning
            tokenized_str.append(self.bos_id)
            images_seq_mask.append(False)

            for message in messages:
                role = message["role"]
                content = message["content"]

                # Check if this is the assistant's turn
                if role == "<|Assistant|>":
                    if not assistant_started:
                        # This is the split point. All tokens added *so far*
                        # are part of the prompt.
                        prompt_token_count = len(tokenized_str)
                        assistant_started = True

                    # Append the EOS token string to the *end* of assistant content
                    content = f"{content.strip()} {self.tokenizer.eos_token}"

                # Split this message's content by the image token
                text_splits = content.split('<image>')

                for i, text_sep in enumerate(text_splits):
                    # Tokenize the text part
                    tokenized_sep = text_encode(self.tokenizer, text_sep, bos=False, eos=False)
                    tokenized_str.extend(tokenized_sep)
                    images_seq_mask.extend([False] * len(tokenized_sep))

                    # If this text is followed by an <image> tag
                    if i < len(text_splits) - 1:
                        if image_idx >= len(images):
                            raise ValueError(
                                f"Data mismatch: Found '<image>' token but no corresponding image."
                            )

                        # Process the image
                        image = images[image_idx]
                        img_list, crop_list, spatial_crop, tok_img, _ = self.process_image(image)

                        images_list.extend(img_list)
                        images_crop_list.extend(crop_list)
                        images_spatial_crop.extend(spatial_crop)

                        # Add image placeholder tokens
                        tokenized_str.extend(tok_img)
                        images_seq_mask.extend([True] * len(tok_img))

                        image_idx += 1 # Move to the next image

            # --- 3. Validation and Final Prep ---
            if image_idx != len(images):
                raise ValueError(
                    f"Data mismatch: Found {len(images)} images but only {image_idx} '<image>' tokens were used."
                )

            # If we never found an assistant message, we're in a weird state
            # (e.g., user-only prompt). We mask everything.
            if not assistant_started:
                print("Warning: No assistant message found in sample. Masking all tokens.")
                prompt_token_count = len(tokenized_str)

            # Prepare image tensors
            images_ori = torch.stack(images_list, dim=0)
            images_spatial_crop_tensor = torch.tensor(images_spatial_crop, dtype=torch.long)

            if images_crop_list:
                images_crop = torch.stack(images_crop_list, dim=0)
            else:
                images_crop = torch.zeros((1, 3, self.base_size, self.base_size), dtype=self.dtype)

            return {
                "input_ids": torch.tensor(tokenized_str, dtype=torch.long),
                "images_seq_mask": torch.tensor(images_seq_mask, dtype=torch.bool),
                "images_ori": images_ori,
                "images_crop": images_crop,
                "images_spatial_crop": images_spatial_crop_tensor,
                "prompt_token_count": prompt_token_count, # This is now accurate
            }

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        """Collate batch of samples"""
        batch_data = []

        # Process each sample
        for feature in features:
            try:
                processed = self.process_single_sample(feature['messages'])
                batch_data.append(processed)
            except Exception as e:
                print(f"Error processing sample: {e}")
                continue

        if not batch_data:
            raise ValueError("No valid samples in batch")

        # Extract lists
        input_ids_list = [item['input_ids'] for item in batch_data]
        images_seq_mask_list = [item['images_seq_mask'] for item in batch_data]
        prompt_token_counts = [item['prompt_token_count'] for item in batch_data]

        # Pad sequences
        input_ids = pad_sequence(input_ids_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        images_seq_mask = pad_sequence(images_seq_mask_list, batch_first=True, padding_value=False)

        # Create labels
        labels = input_ids.clone()

        # Mask padding tokens
        labels[labels == self.tokenizer.pad_token_id] = -100

        # Mask image tokens (model shouldn't predict these)
        labels[images_seq_mask] = -100

        # Mask user prompt tokens when train_on_responses_only=True (only train on assistant responses)
        if self.train_on_responses_only:
            for idx, prompt_count in enumerate(prompt_token_counts):
                if prompt_count > 0:
                    labels[idx, :prompt_count] = -100

        # Create attention mask
        attention_mask = (input_ids != self.tokenizer.pad_token_id).long()

        # Prepare images batch (list of tuples)
        images_batch = []
        for item in batch_data:
            images_batch.append((item['images_crop'], item['images_ori']))

        # Stack spatial crop info
        images_spatial_crop = torch.cat([item['images_spatial_crop'] for item in batch_data], dim=0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "images": images_batch,
            "images_seq_mask": images_seq_mask,
            "images_spatial_crop": images_spatial_crop,
        }

In [40]:
from transformers import Trainer, TrainingArguments
from unsloth import is_bf16_supported
FastVisionModel.for_training(model) # Enable for training!
data_collator = DeepSeekOCRDataCollator(
    tokenizer=tokenizer,
    model = model,
    image_size=640,
    base_size=640,
    crop_mode=True,
    train_on_responses_only=True,
)
trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = data_collator,
    train_dataset = converted_dataset,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 20,
        max_steps = 200,
        # num_train_epoch = 4
        learning_rate = 2e-4,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        fp16 = True,  # True đối với GPU T4
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases
        dataloader_num_workers=4,
        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,

        # CHECKPOINT AN TOÀN
        save_strategy = "steps",
        save_steps = 50,      
        save_total_limit = 2,    
    ),
)

/tmp/ipykernel_47/4155892301.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(


Quá trình Fine-tuning

In [41]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,141 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 737,280 of 3,336,843,520 (0.02% trained)
Unsloth: Not an error, but DeepseekOCRForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([8, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
Unsloth: Will smartly offload gradients to save VRAM!
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([4, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 1

Step,Training Loss
10,1.591300
20,1.270600
30,1.011000
40,0.969600
50,0.847900
60,0.940900
70,0.881500
80,0.816900
90,0.796500
100,0.829100


BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE: 

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


...

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([4, 100, 1280])
BASE: 

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([4, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([4, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([3, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([5, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([4, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 100, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE: 

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


In [52]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [43]:
from unsloth import FastVisionModel
from transformers import AutoModel
import torch

BASE_MODEL = "unsloth/DeepSeek-OCR" 
LORA_DIR = "lora_model"            

model, tokenizer = FastVisionModel.from_pretrained(
    model_name = BASE_MODEL,
    adapter_name = LORA_DIR, 
    load_in_4bit = False,
    auto_model = AutoModel,
    trust_remote_code = True,
)

FastVisionModel.for_inference(model)

print("Model + LoRA loaded")

A new version of the following files was downloaded from https://huggingface.co/unsloth/DeepSeek-OCR:
- deepencoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/unsloth/DeepSeek-OCR:
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/unsloth/DeepSeek-OCR:
- modeling_deepseekocr.py
- deepencoder.py
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and 

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.12.9: Fast Deepseekocr patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


Unsloth: Deepseekocr does not support SDPA - switching to fast eager.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


...

Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at unsloth/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

Model + LoRA loaded


In [44]:
import os
import json

def load_hwdb_test(root_dir):
    samples = []

    # root_dir = .../test_data
    for sub in sorted(os.listdir(root_dir)):
        sub_dir = os.path.join(root_dir, sub)
        if not os.path.isdir(sub_dir):
            continue

        label_path = os.path.join(sub_dir, "label.json")
        if not os.path.exists(label_path):
            print("Missing label.json in", sub_dir)
            continue

        with open(label_path, "r", encoding="utf-8") as f:
            labels = json.load(f)

        for fname, gt_text in labels.items():
            img_path = os.path.join(sub_dir, fname)
            if not os.path.exists(img_path):
                print("Missing image:", img_path)
                continue

            samples.append({
                "image_path": img_path,
                "gt": gt_text
            })

    return samples

In [45]:
import os

def run_ocr(model, tokenizer, image_path):
    prompt = "<image>\nHãy chuyển đổi hình ảnh văn bản viết tay này sang định dạng text tiếng Việt chính xác."

    output_path = "/tmp/ocr"
    os.makedirs(output_path, exist_ok=True)

    result = model.infer(
        tokenizer,
        prompt=prompt,
        image_file=image_path,
        output_path=output_path,
        image_size=640,
        base_size=1024,
        crop_mode=True,
        save_results=False, 
        test_compress=False,
        eval_mode=True,
    )

    print("DEBUG result:", result, type(result))

    if isinstance(result, str):
        return result.strip()
    elif isinstance(result, dict) and "text" in result:
        return result["text"].strip()
    elif hasattr(result, "text"):
        return result.text.strip()
    else:
        return ""

In [47]:
image_file = "/kaggle/input/Vietnamese_Handwritten_OCR/UIT_HWDB_line/UIT_HWDB_line/test_data/250/1.jpg"
pred_text = run_ocr(model, tokenizer, image_file)
print("PRED:", pred_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
DEBUG result:  <class 'str'>
PRED: 


In [48]:
TEST_ROOT = "/kaggle/input/Vietnamese_Handwritten_OCR/UIT_HWDB_line/UIT_HWDB_line/test_data"

test_samples = load_hwdb_test(TEST_ROOT)
print("Total test samples:", len(test_samples))
print(test_samples[:2])

Total test samples: 201
[{'image_path': '/kaggle/input/Vietnamese_Handwritten_OCR/UIT_HWDB_line/UIT_HWDB_line/test_data/250/1.jpg', 'gt': 'Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với'}, {'image_path': '/kaggle/input/Vietnamese_Handwritten_OCR/UIT_HWDB_line/UIT_HWDB_line/test_data/250/2.jpg', 'gt': 'cơ chế " khuyến khích nhà đầu tư thỏa thuận với người sử dụng đất để nhận'}]


Quá trình suy diễn (inference)

In [49]:
from tqdm import tqdm

results = []
for sample in tqdm(test_samples):
    try:
        pred = run_ocr(model, tokenizer, sample["image_path"])
    except Exception as e:
        print("Error:", sample["image_path"], e)
        pred = ""
    print("Gốc:", sample["gt"])
    print("Dự đoán:", pred)
    results.append({"image": sample["image_path"], "gt": sample["gt"], "pred": pred})

  0%|          | 0/201 [00:00<?, ?it/s]

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])


  0%|          | 1/201 [00:00<03:03,  1.09it/s]

DEBUG result:  <class 'str'>
Gốc: Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với
Dự đoán: 
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])


  1%|          | 2/201 [00:01<02:58,  1.11it/s]

...

DEBUG result:  <class 'str'>
Gốc: là thất bại, chỉ là khi thành công - bị - trì - hoãn mà thôi. Cuộc sống vẫn chào đón họ với
Dự đoán: 
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])


100%|█████████▉| 200/201 [02:58<00:00,  1.11it/s]

DEBUG result:  <class 'str'>
Gốc: NV2, NV3. Quan trọng là họ đã nỗ lực hết sức để khẳng định mình. Đó là ý nghĩa vẹn nguyên
Dự đoán: 
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([8, 100, 1280])


100%|██████████| 201/201 [02:59<00:00,  1.12it/s]

DEBUG result:  <class 'str'>
Gốc: của các kỳ thi, và cũng là bản chất của thành công.
Dự đoán: 


In [50]:
for i, r in enumerate(results[:5]):
    print(f"[Mẫu số {i + 1}]")
    print("Gốc     :", repr(r["gt"]))
    print("Dự đoán :", repr(r["pred"]))
    print("-" * 50)

[Mẫu số 1]
Gốc     : 'Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với'
Dự đoán : ''
--------------------------------------------------
[Mẫu số 2]
Gốc     : 'cơ chế " khuyến khích nhà đầu tư thỏa thuận với người sử dụng đất để nhận'
Dự đoán : ''
--------------------------------------------------
[Mẫu số 3]
Gốc     : 'chuyển nhượng hoặc thuê quyền sử dụng đất, nhận góp vốn bằng quyền sử'
Dự đoán : ''
--------------------------------------------------
[Mẫu số 4]
Gốc     : 'dụng đất... " Nhưng tại hội nghị hôm qua, tiếng nói từ các sở tài nguyên'
Dự đoán : ''
--------------------------------------------------
[Mẫu số 5]
Gốc     : '& môi trường lại đều đồng thanh " loại bỏ " điều khoản này ra khỏi dự thảo.'
Dự đoán : ''
--------------------------------------------------


CER

In [51]:
from jiwer import cer

total_cer = 0.0
count = 0

for r in results:
    gt = r["gt"].strip()
    pred = r["pred"].strip()

    if len(gt) == 0:
        continue

    total_cer += cer(gt, pred)
    count += 1

avg_cer = total_cer / count
print(f"CER trên toàn bộ mẫu: {avg_cer:.4f}")


CER trên toàn bộ mẫu: 1.0000
